In [162]:
from collections import defaultdict

In [2]:
wordsFile = open('general/words.txt')
words = []
for word in wordsFile:
    words.append(word.split()[0])

In [3]:
categories = defaultdict()
for category in ['general', 'business', 'entertainment', 'politics', 'sport', 'tech']:
    ngrams = []
    for n in ['uni', 'bi', 'tri', 'four', 'five']:
        if category == 'general':
            filename = f'{category}/{n}grams.txt'
        else:
            filename = f'{category}/{category}_{n}grams.txt'
        
        file = open(filename)
        
        ngramVector = []
        for l in file:
            if n == 'uni':
                ngramVector.append(int(l))
            else:
                ngramVector.append([int(x) for x in l.split()])
        file.close()
        
        # precomputing probabilities for unigrams
        if n == 'uni':
            uniSum = sum(ngramVector)
            ngramVector = [x/uniSum for x in ngramVector]
            
        ngrams.append(ngramVector)
    categories[category] = ngrams

In [404]:
def probsUnigram(categories, category):
    return categories[category][0]

In [405]:
def probsNGram(evidence, categories, category, words):
    n = len(evidence.split())
    corpus = categories[category][n]
    counts = [[words[v[n]], v[n]] for v in corpus if [words.index(e) for e in evidence.split()] == v[:n]]
    countsSum = sum([v[1] for v in counts])
    probabilities = [[v[0], v[1]/countsSum] for v in counts]
    return probabilities

In [406]:
def unigramProbability(word, probVector, words):
    return probVector[words.index(word)]

In [407]:
def nGramProbability(word, probVector):
    prob = [v[1] for v in probVector if v[0] == word]
    if len(prob) == 0:
        return 0
    else:
        return prob[0]

In [408]:
def mixedProb(word, words, uniDist, nGramsDists, lambdas):
    mixed = lambdas[0]*unigramProbability(word, uniDist, words)
    for i in range(0, len(nGramsDists)):
        mixed += lambdas[i+1]*nGramProbability(word, nGramsDists[i])
    return mixed

In [409]:
def predictNextWord(evidence, categories, category, probsUni, lambdas=[0.2]*5):
    evidenceWords = evidence.split()
    n = len(evidenceWords)
    sumRelevantLambdas = sum(lambdas[:n+1])
    normLambda = [x/sumRelevantLambdas for x in lambdas[:n+1]]
    nGramsDists = []
    
    """
    with ProcessPoolExecutor() as executor:
    
        futures = [winprocess.submit(executor, probsNGram, ' '.join(evidenceWords[-i:]), categories, category, words)\
                  for i in range(1, n+1)]

        concurrent.futures.wait(futures)
        nGramsDists = [f.result() for f in futures]
    """
    
    for i in range(1, n+1):
        newEvidence = ' '.join(evidenceWords[-i:])
        nGramsDists.append(probsNGram(newEvidence, categories, category, words))
    
    
    probabilities = []
    """
    with ProcessPoolExecutor() as executor:
        args = categories, category, words, probsUni, nGramsDists, normLambda
        futures = [winprocess.submit(executor, calculateMixedProbs, group, *args)\
                  for group in grouper(words, 1000)]

        concurrent.futures.wait(futures)
        probabilities = list(zip(words, [f.result() for f in futures]))

    """
    for word in words:
        mixed = mixedProb(word, words, probsUni, nGramsDists, normLambda)
        probabilities.append([word, mixed])
    
    
    probabilities = sorted(probabilities, key = lambda x:-x[1])
    return probabilities[0][0]

In [411]:
%%time 
predictNextWord('tv future in the', categories, 'general', \
                        probsUnigram(categories, 'general'))

Wall time: 18 s


'hands'

In [364]:
probsNGram(newEvidence, categories, category, words)

RuntimeError: Failed in object mode pipeline (step: object mode backend)
LLVM IR parsing error
<string>:2692:49: error: expected value token
  %".2219" = call i8* @"PyLong_FromSsize_t"(i64 slice(None, None, None))
                                                ^


In [364]:
probsNGram(newEvidence, categories, category, words)

RuntimeError: Failed in object mode pipeline (step: object mode backend)
LLVM IR parsing error
<string>:2692:49: error: expected value token
  %".2219" = call i8* @"PyLong_FromSsize_t"(i64 slice(None, None, None))
                                                ^


In [426]:
%%time
evidence = 'tv future in the'
for i in range(0, 5):
    newWord = predictNextWord(evidence, categories, 'general', \
                        probsUnigram(categories, 'general'))
    evidence = ' '.join(evidence.split()[-3:] + [newWord])
print(evidence)

of statecontrolled oil firm
Wall time: 18min 58s


In [425]:
%%time
evidence = 'this is one example'
for i in range(0, 5):
    newWord = predictNextWord(evidence, categories, 'general', \
                        probsUnigram(categories, 'general'))
    evidence = ' '.join(evidence.split()[-3:] + [newWord])
print(evidence)

where thieves would set
Wall time: 8min 38s
